# Ch17: Encapsulation (封裝) - 完整解答

本檔案提供 `04-exercises.ipynb` 中所有 15 道習題的完整解答。

**學習重點**:
- 私有屬性的使用方式
- `@property` 裝飾器的完整實作
- Setter 中的驗證邏輯
- 計算屬性與唯讀屬性
- Name Mangling 的應用
- 封裝的最佳實踐

---

## 基礎題解答

### 習題 1: Student 類別

In [ ]:
class Student:
    """學生類別，示範唯讀屬性的實作"""
    
    def __init__(self, name, student_id):
        """
        初始化學生物件
        
        Args:
            name: 學生姓名（公開屬性）
            student_id: 學號（私有屬性，唯讀）
        """
        self.name = name
        self._id = student_id  # 私有屬性，防止外部修改
    
    @property
    def id(self):
        """唯讀屬性：返回學號"""
        return self._id
    
    def __str__(self):
        """字串表示法"""
        return f"Student(name={self.name}, id={self._id})"


# 測試
s = Student("張三", "A12345678")
print(s)  # Student(name=張三, id=A12345678)
print(f"姓名: {s.name}")  # 姓名: 張三
print(f"學號: {s.id}")    # 學號: A12345678

# 可以修改姓名
s.name = "李四"
print(s)  # Student(name=李四, id=A12345678)

# 學號是唯讀的，無法修改
try:
    s.id = "B99999999"  # 會引發 AttributeError
except AttributeError as e:
    print(f"錯誤: {e}")  # can't set attribute

**封裝概念說明**:
- 使用單底線 `_id` 標記為「內部使用」的屬性
- 使用 `@property` 提供唯讀存取介面
- 學號一旦設定就不應該被修改，符合真實世界邏輯

---

### 習題 2: Car 類別 - 速度控制

In [ ]:
class Car:
    """汽車類別，示範屬性驗證與範圍限制"""
    
    def __init__(self, brand):
        """
        初始化汽車物件
        
        Args:
            brand: 汽車品牌
        """
        self.brand = brand
        self._speed = 0  # 初始速度為 0
    
    @property
    def speed(self):
        """取得當前速度"""
        return self._speed
    
    @speed.setter
    def speed(self, value):
        """
        設定速度（必須在 0-200 之間）
        
        Args:
            value: 新的速度值
            
        Raises:
            ValueError: 速度超出範圍時
        """
        if not isinstance(value, (int, float)):
            raise TypeError("速度必須是數字")
        if value < 0:
            raise ValueError("速度不能為負數")
        if value > 200:
            raise ValueError("速度不能超過 200")
        self._speed = value
    
    def accelerate(self, amount):
        """
        加速
        
        Args:
            amount: 加速量
        """
        self.speed = self._speed + amount  # 使用 setter 驗證
    
    def brake(self, amount):
        """
        減速
        
        Args:
            amount: 減速量
        """
        self.speed = self._speed - amount  # 使用 setter 驗證


# 測試
car = Car("Tesla")
print(f"品牌: {car.brand}, 速度: {car.speed}")  # 品牌: Tesla, 速度: 0

# 加速
car.accelerate(50)
print(f"加速後: {car.speed}")  # 加速後: 50

car.accelerate(100)
print(f"再加速: {car.speed}")  # 再加速: 150

# 減速
car.brake(30)
print(f"減速後: {car.speed}")  # 減速後: 120

# 直接設定速度
car.speed = 80
print(f"設定速度: {car.speed}")  # 設定速度: 80

# 測試驗證
try:
    car.speed = 250  # 超過上限
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 速度不能超過 200

try:
    car.brake(100)  # 會導致負數
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 速度不能為負數

**封裝概念說明**:
- Setter 中實作完整的驗證邏輯（型別檢查 + 範圍檢查）
- 公開方法 `accelerate()` 和 `brake()` 內部使用 setter，確保驗證一致性
- 使用者無法設定不合法的速度值

**最佳實踐**:
- 在 setter 中集中處理驗證邏輯
- 所有修改屬性的方法都應該使用 setter（透過 `self.speed = ...`）
- 提供清楚的錯誤訊息

---

### 習題 3: Square 類別 - 正方形

In [ ]:
class Square:
    """正方形類別，示範計算屬性的實作"""
    
    def __init__(self, side):
        """
        初始化正方形物件
        
        Args:
            side: 邊長（必須為正數）
        """
        self.side = side  # 使用 setter 驗證
    
    @property
    def side(self):
        """取得邊長"""
        return self._side
    
    @side.setter
    def side(self, value):
        """
        設定邊長（必須為正數）
        
        Args:
            value: 新的邊長
            
        Raises:
            ValueError: 邊長不是正數時
        """
        if not isinstance(value, (int, float)):
            raise TypeError("邊長必須是數字")
        if value <= 0:
            raise ValueError("邊長必須為正數")
        self._side = value
    
    @property
    def area(self):
        """計算屬性：面積（唯讀）"""
        return self._side ** 2
    
    @property
    def perimeter(self):
        """計算屬性：周長（唯讀）"""
        return self._side * 4


# 測試
sq = Square(5)
print(f"邊長: {sq.side}")        # 邊長: 5
print(f"面積: {sq.area}")        # 面積: 25
print(f"周長: {sq.perimeter}")   # 周長: 20

# 修改邊長，面積和周長自動更新
sq.side = 10
print(f"邊長: {sq.side}")        # 邊長: 10
print(f"面積: {sq.area}")        # 面積: 100
print(f"周長: {sq.perimeter}")   # 周長: 40

# 測試驗證
try:
    sq.side = 0  # 邊長必須為正數
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 邊長必須為正數

try:
    sq.side = -5  # 邊長必須為正數
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 邊長必須為正數

# 面積和周長是唯讀的
try:
    sq.area = 100
except AttributeError as e:
    print(f"錯誤: can't set attribute")  # 無法設定計算屬性

**封裝概念說明**:
- `area` 和 `perimeter` 是計算屬性，不需要儲存，每次存取時動態計算
- 計算屬性只有 getter，沒有 setter（唯讀）
- 當 `side` 改變時，計算屬性會自動反映新值

**最佳實踐**:
- 衍生資料（derived data）應該使用計算屬性，而不是儲存
- 避免資料冗餘和不一致

---

### 習題 4: BankCard 類別 - 信用卡

In [ ]:
class BankCard:
    """信用卡類別，示範資料遮罩與安全性"""
    
    def __init__(self, number):
        """
        初始化信用卡物件
        
        Args:
            number: 卡號（16 位數字字串）
            
        Raises:
            ValueError: 卡號格式不正確時
        """
        if not isinstance(number, str):
            raise TypeError("卡號必須是字串")
        if len(number) != 16 or not number.isdigit():
            raise ValueError("卡號必須是 16 位數字")
        self._number = number
    
    @property
    def number(self):
        """取得完整卡號（唯讀）"""
        return self._number
    
    @property
    def masked_number(self):
        """取得遮罩卡號（只顯示後 4 碼）"""
        return "*" * 12 + self._number[-4:]
    
    def __str__(self):
        """字串表示法（顯示遮罩卡號）"""
        return f"BankCard({self.masked_number})"


# 測試
card = BankCard("1234567890123456")
print(card)  # BankCard(************3456)
print(f"遮罩卡號: {card.masked_number}")  # 遮罩卡號: ************3456

# 可以取得完整卡號（在需要時）
print(f"完整卡號: {card.number}")  # 完整卡號: 1234567890123456

# 測試驗證
try:
    invalid_card = BankCard("12345")  # 長度不足
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 卡號必須是 16 位數字

try:
    invalid_card = BankCard("123456789012345A")  # 包含非數字
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 卡號必須是 16 位數字

# 卡號是唯讀的
try:
    card.number = "9999999999999999"
except AttributeError:
    print("錯誤: 卡號無法修改")  # 錯誤: 卡號無法修改

**封裝概念說明**:
- 敏感資料（卡號）設為私有屬性
- 提供遮罩版本供顯示使用（安全性考量）
- `__str__()` 方法只顯示遮罩卡號，防止意外洩露
- 卡號一旦建立就不能修改（符合真實世界邏輯）

**安全性最佳實踐**:
- 敏感資料應該有遮罩版本
- 預設顯示遮罩版本，完整版本只在必要時存取
- 在建構子中驗證資料格式

---

### 習題 5: Percentage 類別 - 百分比

In [ ]:
class Percentage:
    """百分比類別，示範單位轉換與格式化"""
    
    def __init__(self, value):
        """
        初始化百分比物件
        
        Args:
            value: 百分比值（0-100）
        """
        self.value = value  # 使用 setter 驗證
    
    @property
    def value(self):
        """取得百分比值"""
        return self._value
    
    @value.setter
    def value(self, val):
        """
        設定百分比值（必須在 0-100 之間）
        
        Args:
            val: 新的百分比值
            
        Raises:
            ValueError: 百分比超出範圍時
        """
        if not isinstance(val, (int, float)):
            raise TypeError("百分比必須是數字")
        if val < 0 or val > 100:
            raise ValueError("百分比必須在 0-100 之間")
        self._value = val
    
    @property
    def decimal(self):
        """計算屬性：小數表示（唯讀）"""
        return self._value / 100
    
    def __str__(self):
        """字串表示法"""
        return f"{self._value}%"


# 測試
p = Percentage(75)
print(p)  # 75%
print(f"百分比: {p.value}%")    # 百分比: 75%
print(f"小數: {p.decimal}")     # 小數: 0.75

# 修改百分比
p.value = 50
print(p)  # 50%
print(f"小數: {p.decimal}")     # 小數: 0.5

# 測試邊界值
p.value = 0
print(f"0% = {p.decimal}")      # 0% = 0.0

p.value = 100
print(f"100% = {p.decimal}")    # 100% = 1.0

# 測試驗證
try:
    p.value = 150  # 超過上限
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 百分比必須在 0-100 之間

try:
    p.value = -10  # 低於下限
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 百分比必須在 0-100 之間

**封裝概念說明**:
- 內部儲存百分比格式（0-100）
- 提供小數格式（0-1）的唯讀計算屬性
- `__str__()` 方法提供人類可讀的格式
- 驗證確保百分比在合法範圍內

**設計考量**:
- 選擇最自然的內部表示法（百分比比小數更直觀）
- 透過計算屬性提供其他格式的檢視

---

### 習題 6: Point 類別 - 二維座標

In [ ]:
import math

class Point:
    """二維座標點類別，示範計算屬性與數學運算"""
    
    def __init__(self, x, y):
        """
        初始化座標點物件
        
        Args:
            x: x 座標
            y: y 座標
        """
        self.x = x  # 使用 setter 驗證
        self.y = y
    
    @property
    def x(self):
        """取得 x 座標"""
        return self._x
    
    @x.setter
    def x(self, value):
        """
        設定 x 座標
        
        Args:
            value: 新的 x 座標
        """
        if not isinstance(value, (int, float)):
            raise TypeError("座標必須是數字")
        self._x = value
    
    @property
    def y(self):
        """取得 y 座標"""
        return self._y
    
    @y.setter
    def y(self, value):
        """
        設定 y 座標
        
        Args:
            value: 新的 y 座標
        """
        if not isinstance(value, (int, float)):
            raise TypeError("座標必須是數字")
        self._y = value
    
    @property
    def distance_from_origin(self):
        """計算屬性：距離原點的距離（唯讀）"""
        return math.sqrt(self._x ** 2 + self._y ** 2)
    
    def __str__(self):
        """字串表示法"""
        return f"({self._x}, {self._y})"


# 測試
p = Point(3, 4)
print(p)  # (3, 4)
print(f"x: {p.x}, y: {p.y}")  # x: 3, y: 4
print(f"距離原點: {p.distance_from_origin}")  # 距離原點: 5.0

# 修改座標
p.x = 0
p.y = 0
print(p)  # (0, 0)
print(f"距離原點: {p.distance_from_origin}")  # 距離原點: 0.0

# 3-4-5 直角三角形
p.x = 3
p.y = 4
print(f"距離原點: {p.distance_from_origin}")  # 距離原點: 5.0

# 負座標
p.x = -6
p.y = -8
print(p)  # (-6, -8)
print(f"距離原點: {p.distance_from_origin}")  # 距離原點: 10.0

# 測試驗證
try:
    p.x = "not a number"
except TypeError as e:
    print(f"錯誤: {e}")  # 錯誤: 座標必須是數字

**封裝概念說明**:
- x 和 y 座標可以自由修改（可變物件）
- 距離是計算屬性，當座標改變時自動更新
- 使用畢氏定理計算距離：$\sqrt{x^2 + y^2}$

**數學應用**:
- 封裝幾何計算邏輯
- 使用者不需要知道距離的計算方式
- 計算結果總是與當前座標一致

---

## 中等題解答

### 習題 7: Employee 類別 - 員工薪資

In [ ]:
class Employee:
    """員工類別，示範業務邏輯封裝"""
    
    def __init__(self, name, salary):
        """
        初始化員工物件
        
        Args:
            name: 員工姓名
            salary: 月薪（必須為正數）
        """
        self.name = name
        self.salary = salary  # 使用 setter 驗證
    
    @property
    def salary(self):
        """取得月薪"""
        return self._salary
    
    @salary.setter
    def salary(self, value):
        """
        設定月薪（必須為正數）
        
        Args:
            value: 新的月薪
            
        Raises:
            ValueError: 月薪不是正數時
        """
        if not isinstance(value, (int, float)):
            raise TypeError("月薪必須是數字")
        if value <= 0:
            raise ValueError("月薪必須為正數")
        self._salary = value
    
    @property
    def annual_salary(self):
        """計算屬性：年薪（唯讀）"""
        return self._salary * 12
    
    def give_raise(self, percentage):
        """
        加薪
        
        Args:
            percentage: 加薪百分比（例如 10 表示加薪 10%）
            
        Raises:
            ValueError: 百分比為負數時
        """
        if not isinstance(percentage, (int, float)):
            raise TypeError("百分比必須是數字")
        if percentage < 0:
            raise ValueError("百分比不能為負數")
        
        # 計算加薪金額並更新
        raise_amount = self._salary * (percentage / 100)
        self.salary = self._salary + raise_amount  # 使用 setter


# 測試
emp = Employee("王小明", 50000)
print(f"姓名: {emp.name}")
print(f"月薪: {emp.salary:,.0f}")
print(f"年薪: {emp.annual_salary:,.0f}")
# 姓名: 王小明
# 月薪: 50,000
# 年薪: 600,000

# 加薪 10%
emp.give_raise(10)
print(f"\n加薪 10% 後:")
print(f"月薪: {emp.salary:,.0f}")
print(f"年薪: {emp.annual_salary:,.0f}")
# 加薪 10% 後:
# 月薪: 55,000
# 年薪: 660,000

# 再加薪 5%
emp.give_raise(5)
print(f"\n再加薪 5% 後:")
print(f"月薪: {emp.salary:,.0f}")
print(f"年薪: {emp.annual_salary:,.0f}")
# 再加薪 5% 後:
# 月薪: 57,750
# 年薪: 693,000

# 測試驗證
try:
    emp.salary = -1000  # 薪資不能為負
except ValueError as e:
    print(f"\n錯誤: {e}")  # 錯誤: 月薪必須為正數

**封裝概念說明**:
- 業務邏輯（加薪計算）封裝在方法中
- `give_raise()` 方法使用 setter 更新薪資，確保驗證
- 年薪是計算屬性，總是反映當前月薪

**最佳實踐**:
- 業務操作應該提供語意化的方法（`give_raise()` 比直接修改 `salary` 更清楚）
- 所有修改都透過 setter，維持資料一致性

---

### 習題 8: Fraction 類別 - 分數

In [ ]:
class Fraction:
    """分數類別，示範多屬性驗證與格式化"""
    
    def __init__(self, numerator, denominator):
        """
        初始化分數物件
        
        Args:
            numerator: 分子
            denominator: 分母（不能為 0）
        """
        self.numerator = numerator      # 使用 setter 驗證
        self.denominator = denominator  # 使用 setter 驗證
    
    @property
    def numerator(self):
        """取得分子"""
        return self._numerator
    
    @numerator.setter
    def numerator(self, value):
        """
        設定分子
        
        Args:
            value: 新的分子
        """
        if not isinstance(value, int):
            raise TypeError("分子必須是整數")
        self._numerator = value
    
    @property
    def denominator(self):
        """取得分母"""
        return self._denominator
    
    @denominator.setter
    def denominator(self, value):
        """
        設定分母（不能為 0）
        
        Args:
            value: 新的分母
            
        Raises:
            ValueError: 分母為 0 時
        """
        if not isinstance(value, int):
            raise TypeError("分母必須是整數")
        if value == 0:
            raise ValueError("分母不能為 0")
        self._denominator = value
    
    @property
    def decimal(self):
        """計算屬性：小數表示（唯讀）"""
        return self._numerator / self._denominator
    
    def __str__(self):
        """字串表示法"""
        return f"{self._numerator}/{self._denominator}"


# 測試
f = Fraction(3, 4)
print(f)  # 3/4
print(f"分數: {f}")
print(f"小數: {f.decimal}")  # 小數: 0.75

# 修改分子和分母
f.numerator = 1
f.denominator = 2
print(f"\n修改後: {f}")      # 修改後: 1/2
print(f"小數: {f.decimal}")  # 小數: 0.5

# 負分數
f.numerator = -1
f.denominator = 4
print(f"\n負分數: {f}")      # 負分數: -1/4
print(f"小數: {f.decimal}")  # 小數: -0.25

# 測試驗證
try:
    f.denominator = 0  # 分母不能為 0
except ValueError as e:
    print(f"\n錯誤: {e}")  # 錯誤: 分母不能為 0

try:
    invalid = Fraction(1, 0)  # 建構子也會驗證
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 分母不能為 0

**封裝概念說明**:
- 分子和分母都使用 property 保護
- 分母有額外的驗證（不能為 0）
- 小數值是計算屬性，自動更新

**數學約束**:
- 分數的數學約束（分母≠0）透過封裝強制執行
- 無法建立不合法的分數物件

---

### 習題 9: OnlineOrder 類別 - 線上訂單

In [ ]:
class OnlineOrder:
    """線上訂單類別，示範計算屬性與業務方法"""
    
    def __init__(self, subtotal, shipping_fee=0):
        """
        初始化訂單物件
        
        Args:
            subtotal: 小計金額（必須非負）
            shipping_fee: 運費（必須非負，預設 0）
        """
        self.subtotal = subtotal          # 使用 setter 驗證
        self.shipping_fee = shipping_fee  # 使用 setter 驗證
    
    @property
    def subtotal(self):
        """取得小計金額"""
        return self._subtotal
    
    @subtotal.setter
    def subtotal(self, value):
        """
        設定小計金額（必須非負）
        
        Args:
            value: 新的小計金額
            
        Raises:
            ValueError: 金額為負時
        """
        if not isinstance(value, (int, float)):
            raise TypeError("金額必須是數字")
        if value < 0:
            raise ValueError("金額不能為負數")
        self._subtotal = value
    
    @property
    def shipping_fee(self):
        """取得運費"""
        return self._shipping_fee
    
    @shipping_fee.setter
    def shipping_fee(self, value):
        """
        設定運費（必須非負）
        
        Args:
            value: 新的運費
            
        Raises:
            ValueError: 運費為負時
        """
        if not isinstance(value, (int, float)):
            raise TypeError("運費必須是數字")
        if value < 0:
            raise ValueError("運費不能為負數")
        self._shipping_fee = value
    
    @property
    def total(self):
        """計算屬性：總金額（唯讀）"""
        return self._subtotal + self._shipping_fee
    
    def apply_free_shipping(self):
        """套用免運費優惠"""
        self.shipping_fee = 0


# 測試
order = OnlineOrder(500, 60)
print(f"小計: ${order.subtotal}")
print(f"運費: ${order.shipping_fee}")
print(f"總計: ${order.total}")
# 小計: $500
# 運費: $60
# 總計: $560

# 修改小計
order.subtotal = 800
print(f"\n修改小計後:")
print(f"小計: ${order.subtotal}")
print(f"總計: ${order.total}")
# 修改小計後:
# 小計: $800
# 總計: $860

# 套用免運費
order.apply_free_shipping()
print(f"\n套用免運費後:")
print(f"運費: ${order.shipping_fee}")
print(f"總計: ${order.total}")
# 套用免運費後:
# 運費: $0
# 總計: $800

# 測試驗證
try:
    order.subtotal = -100  # 金額不能為負
except ValueError as e:
    print(f"\n錯誤: {e}")  # 錯誤: 金額不能為負數

**封裝概念說明**:
- 總金額是計算屬性（小計 + 運費），不需要儲存
- `apply_free_shipping()` 提供語意化的業務方法
- 金額驗證確保不會出現負數

**業務邏輯**:
- 電商訂單的常見需求（小計、運費、總計）
- 免運費是常見的促銷活動

---

### 習題 10: ColorRGB 類別 - RGB 顏色

In [ ]:
class ColorRGB:
    """RGB 顏色類別，示範範圍驗證與格式轉換"""
    
    def __init__(self, red, green, blue):
        """
        初始化 RGB 顏色物件
        
        Args:
            red: 紅色值（0-255）
            green: 綠色值（0-255）
            blue: 藍色值（0-255）
        """
        self.red = red      # 使用 setter 驗證
        self.green = green
        self.blue = blue
    
    def _validate_color(self, value):
        """
        驗證顏色值（私有輔助方法）
        
        Args:
            value: 顏色值
            
        Returns:
            驗證後的值
            
        Raises:
            ValueError: 顏色值超出範圍時
        """
        if not isinstance(value, int):
            raise TypeError("顏色值必須是整數")
        if value < 0 or value > 255:
            raise ValueError("顏色值必須在 0-255 之間")
        return value
    
    @property
    def red(self):
        """取得紅色值"""
        return self._red
    
    @red.setter
    def red(self, value):
        """設定紅色值（0-255）"""
        self._red = self._validate_color(value)
    
    @property
    def green(self):
        """取得綠色值"""
        return self._green
    
    @green.setter
    def green(self, value):
        """設定綠色值（0-255）"""
        self._green = self._validate_color(value)
    
    @property
    def blue(self):
        """取得藍色值"""
        return self._blue
    
    @blue.setter
    def blue(self, value):
        """設定藍色值（0-255）"""
        self._blue = self._validate_color(value)
    
    @property
    def hex_code(self):
        """計算屬性：16 進位顏色碼（唯讀）"""
        return f"#{self._red:02X}{self._green:02X}{self._blue:02X}"
    
    def __str__(self):
        """字串表示法"""
        return f"RGB({self._red}, {self._green}, {self._blue}) {self.hex_code}"


# 測試
color = ColorRGB(255, 0, 0)  # 紅色
print(color)  # RGB(255, 0, 0) #FF0000

# 修改顏色
color.red = 0
color.green = 255
print(color)  # RGB(0, 255, 0) #00FF00 (綠色)

color.green = 0
color.blue = 255
print(color)  # RGB(0, 0, 255) #0000FF (藍色)

# 混合顏色
color.red = 255
color.green = 255
color.blue = 0
print(color)  # RGB(255, 255, 0) #FFFF00 (黃色)

# 白色
color.red = 255
color.green = 255
color.blue = 255
print(color)  # RGB(255, 255, 255) #FFFFFF

# 黑色
color.red = 0
color.green = 0
color.blue = 0
print(color)  # RGB(0, 0, 0) #000000

# 測試驗證
try:
    color.red = 256  # 超過上限
except ValueError as e:
    print(f"\n錯誤: {e}")  # 錯誤: 顏色值必須在 0-255 之間

try:
    color.green = -1  # 低於下限
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 顏色值必須在 0-255 之間

**封裝概念說明**:
- 使用私有方法 `_validate_color()` 避免重複驗證邏輯（DRY 原則）
- 三個顏色通道都使用相同的驗證規則
- `hex_code` 計算屬性提供 16 進位格式（常用於網頁）

**最佳實踐**:
- 當多個屬性有相同驗證邏輯時，提取為私有方法
- 格式轉換使用計算屬性，不儲存冗餘資料

---

### 習題 11: ProgressBar 類別 - 進度條

In [ ]:
class ProgressBar:
    """進度條類別，示範相依屬性驗證"""
    
    def __init__(self, total, current=0):
        """
        初始化進度條物件
        
        Args:
            total: 總量（必須為正數）
            current: 當前進度（預設 0，不能超過 total）
        """
        self._total = None     # 先初始化為 None
        self._current = None
        self.total = total     # 先設定 total
        self.current = current # 再設定 current（會驗證是否超過 total）
    
    @property
    def current(self):
        """取得當前進度"""
        return self._current
    
    @current.setter
    def current(self, value):
        """
        設定當前進度（不能超過 total，且必須非負）
        
        Args:
            value: 新的當前進度
            
        Raises:
            ValueError: 進度超出範圍時
        """
        if not isinstance(value, (int, float)):
            raise TypeError("進度必須是數字")
        if value < 0:
            raise ValueError("進度不能為負數")
        if self._total is not None and value > self._total:
            raise ValueError(f"進度不能超過總量 {self._total}")
        self._current = value
    
    @property
    def total(self):
        """取得總量"""
        return self._total
    
    @total.setter
    def total(self, value):
        """
        設定總量（必須為正數）
        
        Args:
            value: 新的總量
            
        Raises:
            ValueError: 總量不是正數時
        """
        if not isinstance(value, (int, float)):
            raise TypeError("總量必須是數字")
        if value <= 0:
            raise ValueError("總量必須為正數")
        self._total = value
        # 如果當前進度超過新的總量，調整為總量
        if self._current is not None and self._current > value:
            self._current = value
    
    @property
    def percentage(self):
        """計算屬性：完成百分比（唯讀）"""
        return (self._current / self._total) * 100
    
    @property
    def is_complete(self):
        """計算屬性：是否完成（唯讀）"""
        return self._current >= self._total
    
    def increment(self, amount=1):
        """
        增加進度
        
        Args:
            amount: 增加量（預設 1）
        """
        self.current = self._current + amount  # 使用 setter 驗證


# 測試
progress = ProgressBar(100, 0)
print(f"進度: {progress.current}/{progress.total}")
print(f"百分比: {progress.percentage:.1f}%")
print(f"是否完成: {progress.is_complete}")
# 進度: 0/100
# 百分比: 0.0%
# 是否完成: False

# 增加進度
progress.increment(25)
print(f"\n進度: {progress.current}/{progress.total}")
print(f"百分比: {progress.percentage:.1f}%")
# 進度: 25/100
# 百分比: 25.0%

progress.increment(50)
print(f"\n進度: {progress.current}/{progress.total}")
print(f"百分比: {progress.percentage:.1f}%")
# 進度: 75/100
# 百分比: 75.0%

# 完成
progress.increment(25)
print(f"\n進度: {progress.current}/{progress.total}")
print(f"百分比: {progress.percentage:.1f}%")
print(f"是否完成: {progress.is_complete}")
# 進度: 100/100
# 百分比: 100.0%
# 是否完成: True

# 測試驗證
try:
    progress.increment(10)  # 超過總量
except ValueError as e:
    print(f"\n錯誤: {e}")  # 錯誤: 進度不能超過總量 100

**封裝概念說明**:
- `current` 和 `total` 是相依屬性（current ≤ total）
- 修改 `total` 時，自動調整 `current`（如果需要）
- `percentage` 和 `is_complete` 是計算屬性
- `increment()` 提供方便的增加進度方法

**相依屬性處理**:
- 初始化時要注意順序（先設定 total，再設定 current）
- Setter 中處理相依關係的一致性

---

## 挑戰題解答

### 習題 12: EmailValidator 類別 - 電子郵件驗證器

In [ ]:
class EmailValidator:
    """電子郵件驗證器類別，示範複雜驗證邏輯"""
    
    def __init__(self, email):
        """
        初始化郵件驗證器物件
        
        Args:
            email: 電子郵件地址
        """
        self.email = email  # 使用 setter 驗證
    
    @property
    def email(self):
        """取得電子郵件地址"""
        return self._email
    
    @email.setter
    def email(self, value):
        """
        設定電子郵件地址（驗證格式）
        
        Args:
            value: 新的電子郵件地址
            
        Raises:
            ValueError: 郵件格式不正確時
        """
        if not isinstance(value, str):
            raise TypeError("Email 必須是字串")
        
        # 基本格式驗證
        if '@' not in value or '.' not in value:
            raise ValueError("Email 必須包含 @ 和 .")
        
        # @ 只能有一個
        if value.count('@') != 1:
            raise ValueError("Email 只能有一個 @")
        
        # @ 前後都必須有字元
        parts = value.split('@')
        if not parts[0] or not parts[1]:
            raise ValueError("@ 前後都必須有字元")
        
        # 域名部分必須包含 .
        domain_part = parts[1]
        if '.' not in domain_part:
            raise ValueError("域名必須包含 .")
        
        # . 不能在開頭或結尾
        if domain_part.startswith('.') or domain_part.endswith('.'):
            raise ValueError(". 不能在域名的開頭或結尾")
        
        self._email = value
    
    @property
    def username(self):
        """計算屬性：使用者名稱（@ 前的部分）"""
        return self._email.split('@')[0]
    
    @property
    def domain(self):
        """計算屬性：域名（@ 後、最後一個 . 前的部分）"""
        domain_part = self._email.split('@')[1]
        return domain_part.rsplit('.', 1)[0]
    
    @property
    def extension(self):
        """計算屬性：副檔名（最後一個 . 後的部分）"""
        domain_part = self._email.split('@')[1]
        return domain_part.rsplit('.', 1)[1]
    
    def is_valid(self):
        """
        檢查 Email 格式是否有效
        
        Returns:
            bool: 如果格式有效返回 True
        """
        try:
            # 如果能成功設定（通過 setter 驗證），就是有效的
            test = self.email
            return True
        except:
            return False


# 測試
email = EmailValidator("user@example.com")
print(f"Email: {email.email}")
print(f"使用者: {email.username}")
print(f"域名: {email.domain}")
print(f"副檔名: {email.extension}")
print(f"有效: {email.is_valid()}")
# Email: user@example.com
# 使用者: user
# 域名: example
# 副檔名: com
# 有效: True

# 測試其他格式
email2 = EmailValidator("john.doe@mail.google.com")
print(f"\nEmail: {email2.email}")
print(f"使用者: {email2.username}")
print(f"域名: {email2.domain}")
print(f"副檔名: {email2.extension}")
# Email: john.doe@mail.google.com
# 使用者: john.doe
# 域名: mail.google
# 副檔名: com

# 測試驗證
try:
    invalid = EmailValidator("notanemail")  # 缺少 @
except ValueError as e:
    print(f"\n錯誤: {e}")  # 錯誤: Email 必須包含 @ 和 .

try:
    invalid = EmailValidator("user@@example.com")  # 兩個 @
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: Email 只能有一個 @

try:
    invalid = EmailValidator("@example.com")  # @ 前沒有字元
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: @ 前後都必須有字元

**封裝概念說明**:
- Setter 中實作完整的 Email 格式驗證邏輯
- 使用計算屬性解析 Email 的各個部分
- `is_valid()` 方法提供額外的驗證檢查

**驗證策略**:
- 在 setter 中進行嚴格驗證，確保只能建立有效的 Email 物件
- 計算屬性假設 Email 已經通過驗證，可以安全解析

**注意**: 這是簡化的 Email 驗證，真實世界的 Email 驗證更複雜（RFC 5322）

---

### 習題 13: Date 類別 - 日期處理

In [ ]:
class Date:
    """日期類別，示範複雜的相依屬性驗證（簡化版）"""
    
    def __init__(self, year, month, day):
        """
        初始化日期物件
        
        Args:
            year: 年份
            month: 月份（1-12）
            day: 日期（1-31，不考慮閏年）
        """
        self._year = None
        self._month = None
        self._day = None
        self.year = year
        self.month = month
        self.day = day
    
    @property
    def year(self):
        """取得年份"""
        return self._year
    
    @year.setter
    def year(self, value):
        """設定年份"""
        if not isinstance(value, int):
            raise TypeError("年份必須是整數")
        if value < 1:
            raise ValueError("年份必須大於 0")
        self._year = value
    
    @property
    def month(self):
        """取得月份"""
        return self._month
    
    @month.setter
    def month(self, value):
        """設定月份（1-12）"""
        if not isinstance(value, int):
            raise TypeError("月份必須是整數")
        if value < 1 or value > 12:
            raise ValueError("月份必須在 1-12 之間")
        self._month = value
    
    @property
    def day(self):
        """取得日期"""
        return self._day
    
    @day.setter
    def day(self, value):
        """設定日期（1-31，簡化版不考慮每月天數）"""
        if not isinstance(value, int):
            raise TypeError("日期必須是整數")
        if value < 1 or value > 31:
            raise ValueError("日期必須在 1-31 之間")
        self._day = value
    
    @property
    def formatted(self):
        """計算屬性：格式化日期（唯讀）"""
        return f"{self._year:04d}-{self._month:02d}-{self._day:02d}"
    
    def is_weekend(self):
        """
        判斷是否為週末（簡化版，假設 2024-01-01 是星期一）
        
        Returns:
            bool: 如果是週末返回 True
        """
        # 簡化計算：假設 2024-01-01 是星期一（day 0）
        # 計算從 2024-01-01 到當前日期的天數
        days_from_start = (self._year - 2024) * 365 + (self._month - 1) * 30 + (self._day - 1)
        day_of_week = days_from_start % 7  # 0=Mon, 1=Tue, ..., 6=Sun
        return day_of_week in [5, 6]  # 5=Sat, 6=Sun
    
    def add_days(self, n):
        """
        增加 n 天（簡化版，不考慮跨月）
        
        Args:
            n: 要增加的天數
        """
        new_day = self._day + n
        if new_day > 31:
            raise ValueError("簡化版不支援跨月計算")
        self.day = new_day
    
    def __str__(self):
        """字串表示法"""
        return self.formatted


# 測試
date = Date(2024, 3, 15)
print(f"日期: {date}")              # 日期: 2024-03-15
print(f"年: {date.year}")
print(f"月: {date.month}")
print(f"日: {date.day}")
print(f"是否週末: {date.is_weekend()}")

# 修改日期
date.day = 20
print(f"\n修改後: {date}")          # 修改後: 2024-03-20

# 增加天數
date.add_days(5)
print(f"增加 5 天: {date}")         # 增加 5 天: 2024-03-25

# 測試週末
date2 = Date(2024, 1, 6)  # 假設是星期六
print(f"\n{date2} 是否週末: {date2.is_weekend()}")

# 測試驗證
try:
    invalid = Date(2024, 13, 1)  # 月份超出範圍
except ValueError as e:
    print(f"\n錯誤: {e}")  # 錯誤: 月份必須在 1-12 之間

try:
    invalid = Date(2024, 1, 32)  # 日期超出範圍
except ValueError as e:
    print(f"錯誤: {e}")  # 錯誤: 日期必須在 1-31 之間

**封裝概念說明**:
- 年、月、日各自有驗證邏輯
- `formatted` 計算屬性提供標準格式（YYYY-MM-DD）
- `is_weekend()` 和 `add_days()` 封裝日期計算邏輯

**簡化說明**:
- 這是教學用的簡化版本，真實的日期處理更複雜
- 未考慮閏年、每月天數、跨月跨年等
- 實務中應使用標準庫 `datetime`

---

### 習題 14: Inventory 類別 - 庫存管理

In [ ]:
class Inventory:
    """庫存管理類別，示範集合型資料的封裝"""
    
    def __init__(self, low_stock_threshold=10):
        """
        初始化庫存物件
        
        Args:
            low_stock_threshold: 低庫存警戒值（預設 10）
        """
        self._items = {}  # 私有字典，儲存商品庫存
        self._low_stock_threshold = low_stock_threshold
    
    def add_stock(self, item, quantity):
        """
        增加庫存
        
        Args:
            item: 商品名稱
            quantity: 增加數量（必須為正數）
            
        Raises:
            ValueError: 數量不是正數時
        """
        if not isinstance(quantity, int) or quantity <= 0:
            raise ValueError("數量必須是正整數")
        
        if item in self._items:
            self._items[item] += quantity
        else:
            self._items[item] = quantity
    
    def remove_stock(self, item, quantity):
        """
        減少庫存
        
        Args:
            item: 商品名稱
            quantity: 減少數量（必須為正數）
            
        Raises:
            ValueError: 數量不合法或庫存不足時
            KeyError: 商品不存在時
        """
        if not isinstance(quantity, int) or quantity <= 0:
            raise ValueError("數量必須是正整數")
        
        if item not in self._items:
            raise KeyError(f"商品 '{item}' 不存在")
        
        if self._items[item] < quantity:
            raise ValueError(f"庫存不足，當前庫存: {self._items[item]}")
        
        self._items[item] -= quantity
        
        # 如果庫存變為 0，移除該商品
        if self._items[item] == 0:
            del self._items[item]
    
    def get_quantity(self, item):
        """
        查詢商品庫存
        
        Args:
            item: 商品名稱
            
        Returns:
            int: 商品庫存量（如果不存在返回 0）
        """
        return self._items.get(item, 0)
    
    @property
    def low_stock_items(self):
        """計算屬性：低於警戒值的商品列表（唯讀）"""
        return [item for item, qty in self._items.items() 
                if qty < self._low_stock_threshold]
    
    @property
    def total_items(self):
        """計算屬性：總商品種類數（唯讀）"""
        return len(self._items)
    
    @property
    def total_quantity(self):
        """計算屬性：總庫存量（唯讀）"""
        return sum(self._items.values())


# 測試
inventory = Inventory(low_stock_threshold=10)

# 增加庫存
inventory.add_stock("蘋果", 50)
inventory.add_stock("香蕉", 30)
inventory.add_stock("橘子", 5)  # 低庫存

print("庫存狀態:")
print(f"蘋果: {inventory.get_quantity('蘋果')}")
print(f"香蕉: {inventory.get_quantity('香蕉')}")
print(f"橘子: {inventory.get_quantity('橘子')}")
# 蘋果: 50
# 香蕉: 30
# 橘子: 5

print(f"\n總商品種類: {inventory.total_items}")
print(f"總庫存量: {inventory.total_quantity}")
# 總商品種類: 3
# 總庫存量: 85

print(f"低庫存商品: {inventory.low_stock_items}")
# 低庫存商品: ['橘子']

# 減少庫存
inventory.remove_stock("蘋果", 20)
print(f"\n售出 20 個蘋果後: {inventory.get_quantity('蘋果')}")
# 售出 20 個蘋果後: 30

# 繼續增加庫存
inventory.add_stock("蘋果", 10)
print(f"補貨 10 個蘋果後: {inventory.get_quantity('蘋果')}")
# 補貨 10 個蘋果後: 40

# 測試驗證
try:
    inventory.remove_stock("橘子", 10)  # 庫存不足
except ValueError as e:
    print(f"\n錯誤: {e}")  # 錯誤: 庫存不足，當前庫存: 5

try:
    inventory.remove_stock("芒果", 5)  # 商品不存在
except KeyError as e:
    print(f"錯誤: {e}")  # 錯誤: '商品 '芒果' 不存在'

**封裝概念說明**:
- 內部使用私有字典 `_items` 儲存資料
- 提供公開方法操作庫存（`add_stock`, `remove_stock`, `get_quantity`）
- 使用者無法直接修改 `_items`，必須透過方法
- 計算屬性提供統計資訊（總商品數、總庫存、低庫存列表）

**業務邏輯**:
- 庫存不能為負數（透過驗證強制執行）
- 低庫存警報（透過計算屬性動態產生）
- 自動清理零庫存商品

**最佳實踐**:
- 集合型資料（字典、列表）應該設為私有
- 提供明確的介面方法來操作資料
- 使用計算屬性而非儲存統計資訊

---

### 習題 15: SecureConfig 類別 - 安全設定

In [ ]:
class SecureConfig:
    """安全設定類別，示範 Name Mangling 的使用"""
    
    def __init__(self):
        """
        初始化安全設定物件
        """
        self.__api_key = None    # 雙底線：Name Mangling
        self.__secret = None     # 雙底線：Name Mangling
    
    def __encrypt(self, value):
        """
        加密方法（簡單版本：反轉字串）
        
        Args:
            value: 要加密的字串
            
        Returns:
            str: 加密後的字串
        """
        if not isinstance(value, str):
            raise TypeError("只能加密字串")
        return value[::-1]  # 反轉字串
    
    def __decrypt(self, value):
        """
        解密方法（簡單版本：反轉字串）
        
        Args:
            value: 要解密的字串
            
        Returns:
            str: 解密後的字串
        """
        if not isinstance(value, str):
            raise TypeError("只能解密字串")
        return value[::-1]  # 反轉字串
    
    def set_api_key(self, key):
        """
        設定 API 金鑰（加密儲存）
        
        Args:
            key: API 金鑰
        """
        if not key:
            raise ValueError("API 金鑰不能為空")
        self.__api_key = self.__encrypt(key)
    
    def get_api_key(self):
        """
        取得 API 金鑰（解密並返回）
        
        Returns:
            str: 解密後的 API 金鑰
            
        Raises:
            ValueError: API 金鑰未設定時
        """
        if self.__api_key is None:
            raise ValueError("API 金鑰尚未設定")
        return self.__decrypt(self.__api_key)
    
    def set_secret(self, secret):
        """
        設定密鑰（加密儲存）
        
        Args:
            secret: 密鑰
        """
        if not secret:
            raise ValueError("密鑰不能為空")
        self.__secret = self.__encrypt(secret)
    
    def verify_secret(self, secret):
        """
        驗證密鑰是否正確
        
        Args:
            secret: 要驗證的密鑰
            
        Returns:
            bool: 密鑰正確返回 True，否則返回 False
        """
        if self.__secret is None:
            return False
        return self.__decrypt(self.__secret) == secret


# 測試
config = SecureConfig()

# 設定 API 金鑰
config.set_api_key("abc123xyz")
print(f"API 金鑰: {config.get_api_key()}")  # API 金鑰: abc123xyz

# 設定密鑰
config.set_secret("mypassword")

# 驗證密鑰
print(f"\n驗證 'mypassword': {config.verify_secret('mypassword')}")  # True
print(f"驗證 'wrongpass': {config.verify_secret('wrongpass')}")    # False

# 嘗試直接存取（會失敗）
try:
    print(config.__api_key)  # AttributeError
except AttributeError:
    print("\n無法直接存取 __api_key（Name Mangling 保護）")

try:
    print(config.__secret)  # AttributeError
except AttributeError:
    print("無法直接存取 __secret（Name Mangling 保護）")

# Name Mangling 後的實際名稱（僅用於示範，不應在實務中使用）
print(f"\n加密的 API 金鑰（內部儲存）: {config._SecureConfig__api_key}")
print(f"加密的密鑰（內部儲存）: {config._SecureConfig__secret}")

# 測試驗證
try:
    empty_config = SecureConfig()
    empty_config.get_api_key()  # 未設定
except ValueError as e:
    print(f"\n錯誤: {e}")  # 錯誤: API 金鑰尚未設定

**封裝概念說明**:
- 使用雙底線 `__api_key` 和 `__secret` 觸發 Name Mangling
- 私有方法 `__encrypt()` 和 `__decrypt()` 封裝加密邏輯
- 敏感資料加密後儲存，只能透過公開方法存取
- Name Mangling 提供額外的保護層

**Name Mangling 說明**:
- `__api_key` 會被重命名為 `_SecureConfig__api_key`
- 這樣可以防止在子類別中意外覆寫
- 也增加了從外部直接存取的難度

**安全性注意**:
- 這是教學用的簡化加密（反轉字串）
- 真實應用應該使用專業的加密庫（如 `cryptography`）
- Name Mangling 不是真正的安全機制，只是命名約定

**最佳實踐**:
- 敏感資料不應該以明文儲存
- 提供驗證方法而非直接返回密碼
- 使用雙底線保護真正不應該被繼承或覆寫的屬性

---

## 總結

### 封裝的核心原則

1. **資料保護**
   - 使用私有屬性（單底線 `_` 或雙底線 `__`）保護內部資料
   - 透過 `@property` 提供受控的存取介面

2. **驗證邏輯**
   - 在 setter 中實作驗證，確保資料一致性
   - 提供明確的錯誤訊息
   - 型別檢查 + 範圍檢查

3. **計算屬性**
   - 衍生資料使用計算屬性，不儲存冗餘
   - 唯讀屬性只提供 getter，沒有 setter

4. **業務方法**
   - 提供語意化的方法封裝業務邏輯
   - 所有修改都透過 setter，維持一致性

5. **Name Mangling**
   - 只在需要防止子類別覆寫時使用雙底線
   - 大多數情況單底線就足夠

### 命名約定總結

| 命名方式 | 用途 | 存取性 |
|:---------|:-----|:-------|
| `name` | 公開屬性 | 完全公開 |
| `_name` | 受保護屬性 | 約定為內部使用 |
| `__name` | 私有屬性 | Name Mangling 保護 |

### 何時使用封裝

- ✅ 需要驗證資料時
- ✅ 需要保護敏感資料時
- ✅ 需要提供唯讀屬性時
- ✅ 需要計算衍生資料時
- ✅ 需要維持資料一致性時

---

**恭喜你完成所有習題！**

你已經掌握了 Python 封裝的核心概念和實作技巧！